In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore") 
import re
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [ ]:
wd=webdriver.Chrome("chromedriver.exe")
wd.get('chrome://settings/')
wd.execute_script('chrome.settingsPrivate.setDefaultZoom(0.4);')
wd.get("https://www.google.ca/")
wd.get("https://www.linkedin.com")

In [ ]:
user="email"
userpass="password"

In [ ]:
username=wd.find_element_by_id("session_key")
username.send_keys(user)
password=wd.find_element_by_id("session_password")
password.send_keys(userpass)

In [ ]:
login_button=wd.find_element_by_class_name("sign-in-form__submit-button")
login_button.click()

In [ ]:
position='data analyst'
loc='canada'
position = position.replace(' ', "%20")
wd.get(f"https://www.linkedin.com/jobs/search/?currentJobId=3087504630&geoId=101174742&keywords={position}&location={loc}&refresh=true")

In [ ]:
no_of_jobs = wd.find_element_by_css_selector('small.display-flex.t-12.t-black--light.t-normal').text
no_of_jobs=int(no_of_jobs.split(' ')[0].replace(',',''))
print(no_of_jobs)
time.sleep(2)

In [ ]:
desc_list = []

for i in range(1, 30):
    # click button to change the job list
    wd.find_element_by_xpath(f'//button[@aria-label="Page {i}"]').click()
    jobs_lists = wd.find_element_by_class_name("jobs-search-results-list")  # here we create a list with jobs
    jobs = jobs_lists.find_elements_by_class_name('jobs-search-results__list-item')  # here we select each job to count
    time.sleep(1)
    # the loop below is for the algorithm to click exactly on the number of jobs that is showing in list
    for j in range(1, len(jobs)+1):
        ## job click
        wd.find_element_by_xpath(f"/html/body/div[5]/div[3]/div[4]/div/div/main/div/section[1]/div/ul/li[{j}]/div/div[1]/div[1]/div[2]/div[1]/a").click()
        ## waiting load 
        time.sleep(1)
        ## select job description
        job_desc = wd.find_element_by_class_name('jobs-description-content__text.t-14.t-normal')
        cards = wd.find_element_by_class_name('jobs-unified-top-card.t-14')
        #get text
        soup = BeautifulSoup(job_desc.get_attribute('outerHTML'), 'html.parser')
        des=soup.text.strip()
        title = cards.find_element_by_class_name('t-24.t-bold').text.strip()
        try:
            company = cards.find_element_by_class_name('ember-view.t-black.t-normal').text.strip()
        except NoSuchElementException:
            company=cards.find_element_by_class_name('jobs-unified-top-card__company-name')
        location = cards.find_element_by_class_name('jobs-unified-top-card__bullet').text.strip()
        try:
            placetype = cards.find_element_by_class_name('jobs-unified-top-card__workplace-type').text.strip()
        except NoSuchElementException:
            placetype = ''
        postdate = cards.find_element_by_class_name('jobs-unified-top-card__posted-date').text
        ## add text to list
        record=(title, company, location, placetype, postdate, des)
        desc_list.append(record)

In [ ]:
""" # Get scroll height
SCROLL_PAUSE_TIME = 0.5
last_height = wd.execute_script("return document.body.scrollHeight")
while True:
# Scroll down to bottom
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = wd.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height """

In [ ]:
df = pd.DataFrame(desc_list)

# deleting useless words
df = df.replace(['\n',
                 '^.*?Expect',
                 '^.*?Qualifications',
                 '^.*?Required',
                 '^.*?expected',
                 '^.*?Responsibilities',
                 '^.*?Requisitos',
                 '^.*?Requirements',
                 '^.*?Qualificações',
                 '^.*?QualificationsRequired1',
                 '^.*?você deve ter:',
                 '^.*?experiência',
                 '^.*?você:',
                 '^.*?Desejável',
                 '^.*?great',
                 '^.*?Looking For',
                 '^.*?ll Need',
                 '^.*?Conhecimento',
                 '^.*?se:',
                 '^.*?habilidades',
                 '^.*?se:',
                 '^.*?REQUISITOS'
                 ], '', regex=True)
df.rename(columns={0:'job_title', 1:'company_name', 2:'location', 3:'workplace_type', 4:'job_posted', 5:'job_desc'}, inplace=True)
df.to_csv('linkedin_jobs.csv')
